In [1]:
# %pip install --upgrade  domolibrary

In [2]:
import domolibrary

domolibrary.__version__

'0.3.39'

In [3]:
import csv

with open("dataset.csv", "r", encoding="utf-8-sig") as file:
    csv_reader = csv.DictReader(file)
    data = [row for row in csv_reader]

[obj.update({"email": obj["email"].replace(" ", "")}) for obj in data]

data[3]

{'email': 'RestaurantManager4@fz.com',
 'Role': 'Restaurant Manager',
 'FranchiseID': '1',
 'StoreID': '7',
 'EmpID': '4',
 'ManagerID': '2'}

# Create User Attributes
User attributes, will be custom properties that we can manage and use to create dynamic pdp policies and groups

In [4]:
properties_to_create = list(
    set(
        [
            key
            for obj in data
            for key in obj.keys()
            if key.lower() not in ["email", "managerid", "empid", "role"]
        ]
    )
)
properties_to_create

['StoreID', 'FranchiseID']

In [5]:
import os
import domolibrary.client.DomoAuth as dmda

token_auth = dmda.DomoTokenAuth(
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
    domo_instance="domo-community",
)

In [6]:
import domolibrary.classes.DomoInstanceConfig as dmic
import domolibrary.classes.DomoInstanceConfig_UserAttribute as dmua

domo_config = dmic.DomoInstanceConfig(auth=token_auth)

await domo_config.user_attributes.get_attributes(
    issuer_type_ls=[dmua.UserAttributes_IssuerType.CUSTOM]
)

[UserAttribute(id='fzStoreID', name='StoreID', description='FZ_POC', issuer_type=<UserAttributes_IssuerType.CUSTOM: 'customer-defined'>, customer_id='mmmm-0012-0200', value_type='STRING', validator='ANY_VALUE', validator_configuration=None, security_voter='FULL_VIS_ADMIN_IDP', custom=True),
 UserAttribute(id='fzFranchiseID', name='FranchiseID', description='FZ_POC', issuer_type=<UserAttributes_IssuerType.CUSTOM: 'customer-defined'>, customer_id='mmmm-0012-0200', value_type='STRING', validator='ANY_VALUE', validator_configuration=None, security_voter='FULL_VIS_ADMIN_IDP', custom=True),
 UserAttribute(id='DomoLibraryStore', name='store_id', description='store ids', issuer_type=<UserAttributes_IssuerType.CUSTOM: 'customer-defined'>, customer_id='mmmm-0012-0200', value_type='STRING', validator='ANY_VALUE', validator_configuration=None, security_voter='FULL_VIS_ADMIN_IDP', custom=True),
 UserAttribute(id='DomoLibraryTestFranchise', name='franchise_id', description='test', issuer_type=<UserA

In [7]:
domo_config.user_attributes

UserAttributes(attributes=[UserAttribute(id='fzStoreID', name='StoreID', description='FZ_POC', issuer_type=<UserAttributes_IssuerType.CUSTOM: 'customer-defined'>, customer_id='mmmm-0012-0200', value_type='STRING', validator='ANY_VALUE', validator_configuration=None, security_voter='FULL_VIS_ADMIN_IDP', custom=True), UserAttribute(id='fzFranchiseID', name='FranchiseID', description='FZ_POC', issuer_type=<UserAttributes_IssuerType.CUSTOM: 'customer-defined'>, customer_id='mmmm-0012-0200', value_type='STRING', validator='ANY_VALUE', validator_configuration=None, security_voter='FULL_VIS_ADMIN_IDP', custom=True), UserAttribute(id='DomoLibraryStore', name='store_id', description='store ids', issuer_type=<UserAttributes_IssuerType.CUSTOM: 'customer-defined'>, customer_id='mmmm-0012-0200', value_type='STRING', validator='ANY_VALUE', validator_configuration=None, security_voter='FULL_VIS_ADMIN_IDP', custom=True), UserAttribute(id='DomoLibraryTestFranchise', name='franchise_id', description='te

In [8]:
import domolibrary.utils.chunk_execution as ce

await ce.gather_with_concurrency(
    *[
        domo_config.user_attributes.upsert(
            attribute_id=f"fz{attribute_name}",
            name=attribute_name,
            description="FZ_POC",
            debug_api=False,
            debug_prn=False,
        )
        for attribute_name in properties_to_create
    ],
    n=10,
)

[
    user_attribute
    for user_attribute in domo_config.user_attributes.attributes
    if user_attribute.id.startswith("fz")
]

[UserAttribute(id='fzStoreID', name='StoreID', description='FZ_POC', issuer_type=<UserAttributes_IssuerType.CUSTOM: 'customer-defined'>, customer_id='mmmm-0012-0200', value_type='STRING', validator='ANY_VALUE', validator_configuration=None, security_voter='FULL_VIS_ADMIN_IDP', custom=True),
 UserAttribute(id='fzFranchiseID', name='FranchiseID', description='FZ_POC', issuer_type=<UserAttributes_IssuerType.CUSTOM: 'customer-defined'>, customer_id='mmmm-0012-0200', value_type='STRING', validator='ANY_VALUE', validator_configuration=None, security_voter='FULL_VIS_ADMIN_IDP', custom=True)]

# create user

In [9]:
data[0]

{'email': 'CEO1@fz.com',
 'Role': 'CEO',
 'FranchiseID': '0',
 'StoreID': '0',
 'EmpID': '1',
 'ManagerID': '1'}

In [10]:
import domolibrary.classes.DomoUser as dmdu

domo_users = await ce.gather_with_concurrency(
    *[
        dmdu.DomoUsers.upsert_user(
            email_address=obj["email"],
            display_name=f"fz{obj['email']}",
            role_id=5,
            auth=token_auth,
            debug_prn=True,
            debug_api=False,
        )
        for obj in data
    ],
    n=6,
)

# update custom properties for users

In [12]:
# %pip install aenum

In [17]:
from domolibrary.classes.DomoUser import UserProperty, UserProperty_Type

# predefined properties we can update
# if a custom property hasn't been added we'll extended it with the aenum library

[member for member in UserProperty_Type]

[<UserProperty_Type.display_name: 'displayName'>,
 <UserProperty_Type.email_address: 'emailAddress'>,
 <UserProperty_Type.phone_number: 'phoneNumber'>,
 <UserProperty_Type.title: 'title'>,
 <UserProperty_Type.department: 'department'>,
 <UserProperty_Type.web_landing_page: 'webLandingPage'>,
 <UserProperty_Type.web_mobile_landing_page: 'webMobileLandingPage'>,
 <UserProperty_Type.role_id: 'roleId'>,
 <UserProperty_Type.employee_id: 'employeeId'>,
 <UserProperty_Type.employee_number: 'employeeNumber'>,
 <UserProperty_Type.hire_date: 'hireDate'>,
 <UserProperty_Type.reports_to: 'reportsTo'>,
 <UserProperty_Type.fzFranchiseID: 'fzFranchiseID'>,
 <UserProperty_Type.fzStoreID: 'fzStoreID'>]

In [18]:
from aenum import extend_enum  # use aenum to extend an existing enum

if "fzFranchiseID" not in UserProperty_Type.__members__:
    extend_enum(UserProperty_Type, "fzFranchiseID", "fzFranchiseID")
if "fzStoreID" not in UserProperty_Type.__members__:
    extend_enum(UserProperty_Type, "fzStoreID", "fzStoreID")

In [19]:
data[0]

{'email': 'CEO1@fz.com',
 'Role': 'CEO',
 'FranchiseID': '0',
 'StoreID': '0',
 'EmpID': '1',
 'ManagerID': '1'}

In [21]:
for obj in data:
    property_ls = [
        UserProperty(UserProperty_Type.fzFranchiseID, obj["FranchiseID"]),
        UserProperty(UserProperty_Type.fzStoreID, obj["StoreID"]),
        UserProperty(UserProperty_Type.employee_id, obj["EmpID"]),
        UserProperty(UserProperty_Type.reports_to, obj["ManagerID"]),
    ]

    print([prop.to_json() for prop in property_ls])

[{'key': 'fzFranchiseID', 'values': ['0']}, {'key': 'fzStoreID', 'values': ['0']}, {'key': 'employeeId', 'values': ['1']}, {'key': 'reportsTo', 'values': ['1']}]
[{'key': 'fzFranchiseID', 'values': ['1']}, {'key': 'fzStoreID', 'values': ['2']}, {'key': 'employeeId', 'values': ['2']}, {'key': 'reportsTo', 'values': ['1']}]
[{'key': 'fzFranchiseID', 'values': ['1']}, {'key': 'fzStoreID', 'values': ['24']}, {'key': 'employeeId', 'values': ['3']}, {'key': 'reportsTo', 'values': ['2']}]
[{'key': 'fzFranchiseID', 'values': ['1']}, {'key': 'fzStoreID', 'values': ['7']}, {'key': 'employeeId', 'values': ['4']}, {'key': 'reportsTo', 'values': ['2']}]
[{'key': 'fzFranchiseID', 'values': ['1']}, {'key': 'fzStoreID', 'values': ['11']}, {'key': 'employeeId', 'values': ['5']}, {'key': 'reportsTo', 'values': ['2']}]
[{'key': 'fzFranchiseID', 'values': ['1']}, {'key': 'fzStoreID', 'values': ['8']}, {'key': 'employeeId', 'values': ['6']}, {'key': 'reportsTo', 'values': ['2']}]
[{'key': 'fzFranchiseID', 